In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [36]:
import pathlib
from PIL import Image
import cv2 as cv

In [37]:
# used for extracting the Posix Paths of the files in the directory "dataset"
dataset_paths = pathlib.Path("dataset")

# dataset of all pairs of image name and paths to the images
dataset = list()
 
# glob() does not check for file inside the subdirectories
dataset_paths = sorted(list(dataset_paths.glob("*.jpeg")))

# iterates through the list of the image paths 
# only keeps relevant image paths and stores (image name : image path) pairs
for pic_path in dataset_paths:
    path = str(pic_path)
    img_name = ""
    for x in path[8:]:
        if(x == "."):
            break
        img_name += x
    img = Image.open(pic_path)
    img = np.asarray(img)
    dataset.append([img_name, img])

In [38]:
dataset

[['0',
  array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         ...,
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]], dtype=uint8)],
 ['1',
  array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
        

In [39]:

# image = cv.imread('dataset/1.jpeg')

# gray = cv.cvtColor(dataset[1][1], cv.COLOR_BGR2GRAY) 
def togray(x):
    image = dataset[x][1]; gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) 
    dataset[x][1] =np.asarray(gray).reshape(-1,)
# cv.imshow('1',gray)
# cv.waitKey(0)


In [40]:
for i in range(36):
    togray(i)

In [27]:
dataset[1][1]

(160000,)

In [51]:
for i in range(len(dataset)):
    dataset[i] = [ord(dataset[i][0]), dataset[i][1].reshape(-1,)]

In [52]:
dataset[3][1]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [42]:
data = pd.DataFrame(dataset, columns = ["Symbol", "Hand_Sign"])

In [43]:
data.iloc[2,1]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [44]:
data.iloc[3,1].shape

(160000,)

In [17]:
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# model.fit([ [x] for x in list(data.iloc[:, 0])], list(data.iloc[: , 1]))

In [45]:
print("Image shape : ", data.iloc[0,1].shape)

Image shape :  (160000,)


In [57]:
import tensorflow as tf

In [65]:
model = tf.keras.models.Sequential({
    tf.keras.layers.Input(shape=(1,)),
    tf.keras.layers.Dense(1, activation = "relu"),
    tf.keras.layers.Dense(36, activation = "relu"),
    tf.keras.layers.Dense(48000),
    tf.keras.layers.Dense(1)
})

In [66]:
model.compile(optimizer='adam',
              loss = 'MeanSquaredError',
              metrics=['accuracy'])

In [67]:
X_train = tf.convert_to_tensor(data.iloc[:,0])
X_train

<tf.Tensor: shape=(36,), dtype=int64, numpy=
array([ 48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  97,  98,  99,
       100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122])>

In [68]:
# l = [(data.iloc[:,1])[x].reshape(16000,-1) for x in range(36)]
l = list(data.iloc[:,1])
y_train = tf.convert_to_tensor(l)
y_train

<tf.Tensor: shape=(36, 480000), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [69]:
model.fit(X_train, y_train, epochs = 5)

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5374 - loss: 7029.3589
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0087 - loss: 5978.7998
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0087 - loss: 5210.2144 
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0087 - loss: 4784.5132 
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0087 - loss: 4955.3926


In [76]:
model.predict(tf.convert_to_tensor([48]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([[31.264902]], dtype=float32)

In [10]:
def showsign(x):
    cv.imshow('1',dataset[x][1].reshape(400,400))
    cv.waitKey(1)
    # plt.title(f"Hand Sign for \'{dataset[x][0]}\'")

In [1]:
showsign(1)

NameError: name 'showsign' is not defined

In [ ]:
keys = dataset[:][0]
values = dataset[:][1]
image_map = dict(zip(keys, values))

[['0', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['1', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['2', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['3', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['4', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['5', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['6', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['7', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['8', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['9', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['a', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['b', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['c', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['d', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['e', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['f', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['g', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['h', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
 ['i', array([0, 0, 0, ..., 0, 0, 0], dtype=ui

In [41]:
print(dataset[0])

['0', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)]


In [46]:
data.to_numpy()

array([['0', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['1', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['2', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['3', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['4', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['5', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['6', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['7', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['8', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['9', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['a', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['b', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['c', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['d', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['e', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['f', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)],
       ['g', array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [47]:
np.save('dataset/image_set', data.to_numpy())

In [49]:
data = data.to_numpy()

In [55]:
data[:,1]

array([array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
       array